In [1]:
%load_ext autoreload
%autoreload 2

import sys, os, time, re, datetime

import numpy as np
import tensorflow as tf
from tensorflow import keras
print(f"Using Tensorflow v{tf.__version__}")

# For managing relative imports from notebook
if '..' not in sys.path: sys.path.append('..')

import config.config as dfc
import deepfake.dfutillib as df
import deepfake.modelutil as mutil

Using Tensorflow v2.1.0


In [2]:
# ------------------------- Model Instantiation -------------------------

mwrapper = mutil.ModelWrapper()
model = mwrapper.init_encdec_network(traindiff=False, trainfake=False)
model.load_weights(f'{dfc.MODEL_STORE}/model.60-0.42.hdf5')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 360, 640, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 175, 158, 32) 11648       input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 87, 79, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 85, 77, 64)   18496       max_pooling2d[0][0]              
______________________________________________________________________________________________

In [3]:
# Model data loaders wrapped in tf.data Dataset
testloader = mutil.ModelLoader(split='test')
testdataset = tf.data.Dataset.from_generator(generator=testloader.test_loader, 
    output_types=tf.uint8, output_shapes=(None, 360, 640, 3))

# holdoutloader = mutil.ModelLoader(split='holdout')
# holdoutdataset = tf.data.Dataset.from_generator(generator=holdoutloader.holdout_loader, 
#     output_types=tf.uint8, output_shapes=(None, 360, 640, 3))


In [4]:
framepredicts = model.predict(x=testdataset, steps=testloader.epochsz, verbose=1)
# framepredicts = model.predict(x=holdoutdataset, steps=holdoutloader.epochsz, verbose=1)

400/400 [==============================] - 1020s 3s/step


In [5]:
framepredicts = np.squeeze(framepredicts)
with open('./submission.csv', 'w') as f_handle:
#{
    f_handle.write('filename,label\n')
    for vidx, fstep in enumerate(range(0, len(framepredicts), 60)):
    #{    
        ireal = np.argmin(framepredicts[fstep:fstep+60], axis=0)
        ifake = np.argmax(framepredicts[fstep:fstep+60], axis=0)
        vidavg = np.average(framepredicts[fstep:fstep+60])
        bestpred = (framepredicts[fstep+ireal] if vidavg < 0.5 
                    else framepredicts[fstep+ifake])
        
        f_handle.write(f'{testloader.videostack[vidx]},{bestpred}\n')
        #f_handle.write(f'{videostack[vidx]},{bestpred}\n')
    #}
#}

In [ ]:
# import re

# regex = r'model.*-(\d\.\d\d).*hdf5'
# filterkey = lambda mname: re.match(regex, mname)
# sortkey = lambda mname: float(re.match(regex, mname).group(1))

# sorted(filter(filterkey, os.listdir(dfc.MODEL_STORE)), key=sortkey)